In [1]:
import cv2
import mediapipe as mp
import pyautogui
import math
from enum import IntEnum
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
from google.protobuf.json_format import MessageToDict
import screen_brightness_control as sbcontrol

In [2]:
# To create GUI
import tkinter as tk
from PIL import ImageTk, Image
pyautogui.FAILSAFE = False
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
# Gesture Encodings 
class Gest(IntEnum):
    # Binary Encoded
    FIST = 0
    PINKY = 1
    RING = 2
    MID = 4
    LAST3 = 7
    INDEX = 8
    FIRST2 = 12
    LAST4 = 15
    THUMB = 16    
    PALM = 31  
    # Extra Mappings
    V_GEST = 33
    TWO_FINGER_CLOSED = 34
    PINCH_MAJOR = 35
    PINCH_MINOR = 36

# Multi-handedness Labels
class HLabel(IntEnum):
    MINOR = 0
    MAJOR = 1

In [3]:
class HandRecog:  
    def __init__(self, hand_label):
        self.finger = 0
        self.ori_gesture = Gest.PALM
        self.prev_gesture = Gest.PALM
        self.frame_count = 0
        self.hand_result = None
        self.hand_label = hand_label
        
    def update_hand_result(self, hand_result):
        self.hand_result = hand_result
        
    def get_signed_dist(self, point):
        sign = -1
        if self.hand_result.landmark[point[0]].y < self.hand_result.landmark[point[1]].y:
            sign = 1
        dist = (self.hand_result.landmark[point[0]].x - self.hand_result.landmark[point[1]].x)**2
        dist += (self.hand_result.landmark[point[0]].y - self.hand_result.landmark[point[1]].y)**2
        dist = math.sqrt(dist)
        return dist*sign 
        
    def get_dist(self, point):
        dist = (self.hand_result.landmark[point[0]].x - self.hand_result.landmark[point[1]].x)**2
        dist += (self.hand_result.landmark[point[0]].y - self.hand_result.landmark[point[1]].y)**2
        dist = math.sqrt(dist)
        return dist  
        
    def get_dz(self,point):
        return abs(self.hand_result.landmark[point[0]].z - self.hand_result.landmark[point[1]].z)   
        
    # Function to find Gesture Encoding using current finger_state.
    # Finger_state: 1 if finger is open, else 0
    def set_finger_state(self):
        if self.hand_result == None:
            return
        points = [[8,5,0],[12,9,0],[16,13,0],[20,17,0]]
        self.finger = 0
        self.finger = self.finger | 0 #thumb
        for idx,point in enumerate(points):            
            dist = self.get_signed_dist(point[:2])
            dist2 = self.get_signed_dist(point[1:])          
            try:
                ratio = round(dist/dist2,1)
            except:
                ratio = round(dist/0.01,1)
            self.finger = self.finger << 1
            if ratio > 0.5 :
                self.finger = self.finger | 1
                
    # Handling Fluctations due to noise
    def get_gesture(self):
        if self.hand_result == None:
            return Gest.PALM
    
        current_gesture = Gest.PALM
    
        # Existing gesture detection logic
        if self.finger in [Gest.LAST3, Gest.LAST4] and self.get_dist([8,4]) < 0.05:
            if self.hand_label == HLabel.MINOR:
                current_gesture = Gest.PINCH_MINOR
            else:
                current_gesture = Gest.PINCH_MAJOR
        elif Gest.FIRST2 == self.finger:
            point = [[8,12], [5,9]]
            dist1 = self.get_dist(point[0])
            dist2 = self.get_dist(point[1])
            ratio = dist1/dist2

            if ratio > 1.7:
                current_gesture = Gest.V_GEST
            else:
                if self.get_dz([8,12]) < 0.1:
                    current_gesture = Gest.TWO_FINGER_CLOSED
                else:
                    current_gesture = Gest.MID
        else:
            current_gesture = self.finger
    
        # Enhanced smoothing mechanism
        if not hasattr(self, 'gesture_history'):
            self.gesture_history = []
            self.gesture_confidence = {}
            self.min_confidence_threshold = 3
            self.history_size = 7
        
        # Add current gesture to history
        self.gesture_history.append(current_gesture)
    
        # Keep only recent history
        if len(self.gesture_history) > self.history_size:
            self.gesture_history.pop(0)
        
        # Count occurrences of each gesture in recent history
        gesture_counts = {}
        for gesture in self.gesture_history:
            gesture_counts[gesture] = gesture_counts.get(gesture, 0) + 1
        
        # Find the most frequent gesture
        most_frequent_gesture = max(gesture_counts, key=gesture_counts.get)
        max_count = gesture_counts[most_frequent_gesture]
        
        # Update confidence for the most frequent gesture
        if most_frequent_gesture in self.gesture_confidence:
            self.gesture_confidence[most_frequent_gesture] += 1
        else:
            self.gesture_confidence[most_frequent_gesture] = 1
        
        # Decay confidence for other gestures
        for gesture in list(self.gesture_confidence.keys()):
            if gesture != most_frequent_gesture:
                self.gesture_confidence[gesture] = max(0, self.gesture_confidence[gesture] - 0.5)
                if self.gesture_confidence[gesture] == 0:
                    del self.gesture_confidence[gesture]
        
        # Determine if we should change the output gesture
        if (max_count >= 3 and  # Gesture appears in at least 3 recent frames
            self.gesture_confidence.get(most_frequent_gesture, 0) >= self.min_confidence_threshold):
            
            # Additional smoothing for similar gestures
            if hasattr(self, 'ori_gesture') and self.ori_gesture != most_frequent_gesture:
                # Check if the new gesture is "similar" to avoid rapid switching
                if self._gestures_are_similar(self.ori_gesture, most_frequent_gesture):
                    # Require higher confidence for similar gestures
                    if self.gesture_confidence.get(most_frequent_gesture, 0) >= self.min_confidence_threshold + 2:
                        self.ori_gesture = most_frequent_gesture
                else:
                    self.ori_gesture = most_frequent_gesture
            else:
                self.ori_gesture = most_frequent_gesture
        
        # Initialize ori_gesture if not set
        if not hasattr(self, 'ori_gesture'):
            self.ori_gesture = Gest.PALM
        
        return self.ori_gesture

    def _gestures_are_similar(self, gesture1, gesture2):
        """Helper method to determine if two gestures are similar"""
        # Define similar gesture groups
        similar_groups = [
            [Gest.V_GEST, Gest.FIRST2, Gest.TWO_FINGER_CLOSED],
            [Gest.PINCH_MAJOR, Gest.PINCH_MINOR],
            [Gest.LAST3, Gest.LAST4]
        ]
        
        for group in similar_groups:
            if gesture1 in group and gesture2 in group:
                return True
        return False

    # Alternative simpler smoothing approach
    def get_gesture_simple_smooth(self):
        if self.hand_result == None:
            return Gest.PALM
        
        current_gesture = Gest.PALM
        
        # Your existing gesture detection logic
        if self.finger in [Gest.LAST3, Gest.LAST4] and self.get_dist([8,4]) < 0.05:
            if self.hand_label == HLabel.MINOR:
                current_gesture = Gest.PINCH_MINOR
            else:
                current_gesture = Gest.PINCH_MAJOR
        elif Gest.FIRST2 == self.finger:
            point = [[8,12], [5,9]]
            dist1 = self.get_dist(point[0])
            dist2 = self.get_dist(point[1])
            ratio = dist1/dist2
            if ratio > 1.7:
                current_gesture = Gest.V_GEST
            else:
                if self.get_dz([8,12]) < 0.1:
                    current_gesture = Gest.TWO_FINGER_CLOSED
                else:
                    current_gesture = Gest.MID
        else:
            current_gesture = self.finger
        
        # Simple smoothing with adjustable threshold
        if current_gesture == self.prev_gesture:
            self.frame_count += 1
        else:
            self.frame_count = 0
        
        self.prev_gesture = current_gesture
        
        # Adjustable smoothing threshold based on gesture type
        threshold = self._get_smoothing_threshold(current_gesture)
        
        if self.frame_count > threshold:
            self.ori_gesture = current_gesture
        
        return self.ori_gesture

    def _get_smoothing_threshold(self, gesture):
        """Get different smoothing thresholds for different gestures"""
        # More sensitive gestures need higher thresholds
        sensitive_gestures = [Gest.PINCH_MAJOR, Gest.PINCH_MINOR, Gest.TWO_FINGER_CLOSED]
        
        if gesture in sensitive_gestures:
            return 6  # Higher threshold for sensitive gestures
        elif gesture == Gest.PALM:
            return 2  # Lower threshold for palm (default state)
        else:
            return 4  # Standard threshold for other gestures

In [4]:
class SmoothControlData:
    def __init__(self):
        self.last_brightness_change = 0
        self.last_volume_change = 0
        self.last_scroll_change = 0
        self.brightness_cooldown = 0.08  # 80ms cooldown
        self.volume_cooldown = 0.08     # 80ms cooldown
        self.scroll_cooldown = 0.1      # 100ms cooldown
        
        # Smoothing buffers
        self.brightness_history = []
        self.volume_history = []
        self.scroll_history = []
        self.history_size = 3
        
        # Last applied values to prevent micro-adjustments
        self.last_applied_brightness = None
        self.last_applied_volume = None

# Initialize the smooth control data (add this where you initialize your Controller)
smooth_control = SmoothControlData()

In [5]:
def set_finger_state(self):
    """Updates finger states"""
    if self.hand_result == None:
        return

    points = self.hand_result.landmark
    self.finger = int('1' * 5, 2)
    
    # Palm detection - check if all fingers are open
    if points[2].y < points[3].y:       # Index
        self.finger = self.finger & 29
    if points[6].y < points[7].y:       # Middle
        self.finger = self.finger & 27
    if points[10].y < points[11].y:     # Ring
        self.finger = self.finger & 23
    if points[14].y < points[15].y:     # Pinky
        self.finger = self.finger & 15
    if points[18].x > points[4].x:      # Thumb
        self.finger = self.finger & 31

In [6]:
import time
class Controller:
    last_right_click=0
    tx_old = 0
    ty_old = 0
    britness_val=50
    trial = True
    flag = False
    grabflag = False
    pinchmajorflag = False
    pinchminorflag = False
    pinchstartxcoord = None
    pinchstartycoord = None
    pinchdirectionflag = None
    prevpinchlv = 0
    pinchlv = 0
    framecount = 0
    prev_hand = None
    pinch_threshold = 0.3
    
    def getpinchylv(hand_result):
        dist = round((Controller.pinchstartycoord - hand_result.landmark[8].y)*10,1)
        return dist
    def getpinchxlv(hand_result):
        dist = round((hand_result.landmark[8].x - Controller.pinchstartxcoord)*10,1)
        return dist   
    def changesystembrightness():
        import time
        current_time = time.time()
        
        # Check cooldown to prevent rapid changes
        if current_time - smooth_control.last_brightness_change < smooth_control.brightness_cooldown:
            return
        
        # Add current pinch level to history for smoothing
        smooth_control.brightness_history.append(Controller.pinchlv)
        
        # Keep only recent history
        if len(smooth_control.brightness_history) > smooth_control.history_size:
            smooth_control.brightness_history.pop(0)
        
        # Calculate smoothed pinch level
        if len(smooth_control.brightness_history) >= 2:
            # Use weighted average - give more weight to recent values
            weights = [0.3, 0.7] if len(smooth_control.brightness_history) == 2 else [0.2, 0.3, 0.5]
            smoothed_pinchlv = sum(val * weight for val, weight in 
                                zip(smooth_control.brightness_history, weights))
        else:
            smoothed_pinchlv = Controller.pinchlv
        
        # Only proceed if the change is significant enough
        if abs(smoothed_pinchlv) < 0.02:  # Ignore very small changes
            return
        
        try:
            currentBrightnessLv = sbcontrol.get_brightness()[0] / 100.0  # Extract the first element
            newBrightnessLv = currentBrightnessLv + (smoothed_pinchlv / 50.0)
            
            # Clamp values
            if newBrightnessLv > 1.0:
                newBrightnessLv = 1.0
            elif newBrightnessLv < 0.0:
                newBrightnessLv = 0.0
            
            # Only apply if change is significant enough (prevent micro-adjustments)
            if smooth_control.last_applied_brightness is None or \
            abs(newBrightnessLv - smooth_control.last_applied_brightness) > 0.03:
                
                sbcontrol.fade_brightness(int(100 * newBrightnessLv), start=sbcontrol.get_brightness()[0])  # Extract the first element
                smooth_control.last_applied_brightness = newBrightnessLv
                smooth_control.last_brightness_change = current_time
                
        except Exception as e:
            print(f"Brightness control error: {e}")   
    def changesystemvolume():
        import time
        current_time = time.time()
        
        # Check cooldown to prevent rapid changes
        if current_time - smooth_control.last_volume_change < smooth_control.volume_cooldown:
            return
        
        # Add current pinch level to history for smoothing
        smooth_control.volume_history.append(Controller.pinchlv)
        
        # Keep only recent history
        if len(smooth_control.volume_history) > smooth_control.history_size:
            smooth_control.volume_history.pop(0)
        
        # Calculate smoothed pinch level
        if len(smooth_control.volume_history) >= 2:
            # Use weighted average - give more weight to recent values
            weights = [0.3, 0.7] if len(smooth_control.volume_history) == 2 else [0.2, 0.3, 0.5]
            smoothed_pinchlv = sum(val * weight for val, weight in 
                                zip(smooth_control.volume_history, weights))
        else:
            smoothed_pinchlv = Controller.pinchlv
        
        # Only proceed if the change is significant enough
        if abs(smoothed_pinchlv) < 0.02:  # Ignore very small changes
            return
        
        try:
            devices = AudioUtilities.GetSpeakers()
            interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
            volume = cast(interface, POINTER(IAudioEndpointVolume))
            currentVolumeLv = volume.GetMasterVolumeLevelScalar()
            newVolumeLv = currentVolumeLv + (smoothed_pinchlv/50.0)
            
            # Clamp values
            if newVolumeLv > 1.0:
                newVolumeLv = 1.0
            elif newVolumeLv < 0.0:
                newVolumeLv = 0.0
            
            # Only apply if change is significant enough (prevent micro-adjustments)
            if smooth_control.last_applied_volume is None or \
            abs(newVolumeLv - smooth_control.last_applied_volume) > 0.03:
                
                volume.SetMasterVolumeLevelScalar(newVolumeLv, None)
                smooth_control.last_applied_volume = newVolumeLv
                smooth_control.last_volume_change = current_time
                
        except Exception as e:
            print(f"Volume control error: {e}")

    def scrollVertical():
        import time
        current_time = time.time()
        
        # Check cooldown to prevent rapid scrolling
        if current_time - smooth_control.last_scroll_change < smooth_control.scroll_cooldown:
            return
        
        # Add current pinch level to history for smoothing
        smooth_control.scroll_history.append(Controller.pinchlv)
        
        # Keep only recent history
        if len(smooth_control.scroll_history) > smooth_control.history_size:
            smooth_control.scroll_history.pop(0)
        
        # Calculate smoothed pinch level
        if len(smooth_control.scroll_history) >= 2:
            weights = [0.3, 0.7] if len(smooth_control.scroll_history) == 2 else [0.2, 0.3, 0.5]
            smoothed_pinchlv = sum(val * weight for val, weight in 
                                zip(smooth_control.scroll_history, weights))
        else:
            smoothed_pinchlv = Controller.pinchlv
        
        # Only proceed if the change is significant enough
        if abs(smoothed_pinchlv) < 0.1:  # Higher threshold for scrolling
            return
        
        try:
            # Variable scroll amount based on pinch level
            scroll_amount = int(120 * abs(smoothed_pinchlv) / 0.5)  # Scale scroll amount
            scroll_amount = max(60, min(240, scroll_amount))  # Clamp between 60 and 240
            
            pyautogui.scroll(scroll_amount if smoothed_pinchlv > 0.0 else -scroll_amount)
            smooth_control.last_scroll_change = current_time
            
        except Exception as e:
            print(f"Scroll control error: {e}")

    def scrollHorizontal():
        import time
        current_time = time.time()
        
        # Check cooldown to prevent rapid scrolling
        if current_time - smooth_control.last_scroll_change < smooth_control.scroll_cooldown:
            return
        
        # Add current pinch level to history for smoothing
        smooth_control.scroll_history.append(Controller.pinchlv)
        
        # Keep only recent history
        if len(smooth_control.scroll_history) > smooth_control.history_size:
            smooth_control.scroll_history.pop(0)
        
        # Calculate smoothed pinch level
        if len(smooth_control.scroll_history) >= 2:
            weights = [0.3, 0.7] if len(smooth_control.scroll_history) == 2 else [0.2, 0.3, 0.5]
            smoothed_pinchlv = sum(val * weight for val, weight in 
                                zip(smooth_control.scroll_history, weights))
        else:
            smoothed_pinchlv = Controller.pinchlv
        
        # Only proceed if the change is significant enough
        if abs(smoothed_pinchlv) < 0.1:  # Higher threshold for scrolling
            return
        
        try:
            # Variable scroll amount based on pinch level
            scroll_amount = int(120 * abs(smoothed_pinchlv) / 0.5)  # Scale scroll amount
            scroll_amount = max(60, min(240, scroll_amount))  # Clamp between 60 and 240
            
            pyautogui.keyDown('shift')
            pyautogui.keyDown('ctrl')
            pyautogui.scroll(-scroll_amount if smoothed_pinchlv > 0.0 else scroll_amount)
            pyautogui.keyUp('ctrl')
            pyautogui.keyUp('shift')
            smooth_control.last_scroll_change = current_time
            
        except Exception as e:
            print(f"Horizontal scroll control error: {e}")

    # Optional: Add this function to reset smoothing data when needed
    def reset_smooth_controls():
        """Reset all smoothing data - useful when switching between different control modes"""
        smooth_control.brightness_history.clear()
        smooth_control.volume_history.clear()
        smooth_control.scroll_history.clear()
        smooth_control.last_applied_brightness = None
        smooth_control.last_applied_volume = None    # Locate Hand to get Cursor Position
    # Stabilize cursor by Dampening
    def get_position(hand_result):
        point = 9
        position = [hand_result.landmark[point].x ,hand_result.landmark[point].y]
        sx,sy = pyautogui.size()
        x_old,y_old = pyautogui.position()
        x = int(position[0]*sx)
        y = int(position[1]*sy)
        if Controller.prev_hand is None:
            Controller.prev_hand = x,y
        delta_x = x - Controller.prev_hand[0]
        delta_y = y - Controller.prev_hand[1]
        distsq = delta_x**2 + delta_y**2
        ratio = 1
        Controller.prev_hand = [x,y]
        if distsq <= 25:
            ratio = 0
        elif distsq <= 900:
            ratio = 0.07 * (distsq ** (1/2))
        else:
            ratio = 2.1
        x , y = x_old + delta_x*ratio , y_old + delta_y*ratio
        return (x,y)
    def pinch_control_init(hand_result):
        Controller.pinchstartxcoord = hand_result.landmark[8].x
        Controller.pinchstartycoord = hand_result.landmark[8].y
        Controller.pinchlv = 0
        Controller.prevpinchlv = 0
        Controller.framecount = 0
    # Hold final position for 5 frames to change status
    def pinch_control(hand_result, controlHorizontal, controlVertical):
        if Controller.framecount == 5:
            Controller.framecount = 0
            Controller.pinchlv = Controller.prevpinchlv

            if Controller.pinchdirectionflag == True:
                controlHorizontal() #x

            elif Controller.pinchdirectionflag == False:
                controlVertical() #y
        lvx =  Controller.getpinchxlv(hand_result)
        lvy =  Controller.getpinchylv(hand_result)           
        if abs(lvy) > abs(lvx) and abs(lvy) > Controller.pinch_threshold:
            Controller.pinchdirectionflag = False
            if abs(Controller.prevpinchlv - lvy) < Controller.pinch_threshold:
                Controller.framecount += 1
            else:
                Controller.prevpinchlv = lvy
                Controller.framecount = 0

        elif abs(lvx) > Controller.pinch_threshold:
            Controller.pinchdirectionflag = True
            if abs(Controller.prevpinchlv - lvx) < Controller.pinch_threshold:
                Controller.framecount += 1
            else:
                Controller.prevpinchlv = lvx
                Controller.framecount = 0
                
    def handle_controls(gesture, hand_result):        
        x,y = None,None
        if gesture != Gest.PALM :
            x,y = Controller.get_position(hand_result)     
        # flag reset
        if gesture != Gest.FIST and Controller.grabflag:
            Controller.grabflag = False
            pyautogui.mouseUp(button = "left")
        if gesture != Gest.PINCH_MAJOR and Controller.pinchmajorflag:
            Controller.pinchmajorflag = False
        if gesture != Gest.PINCH_MINOR and Controller.pinchminorflag:
            Controller.pinchminorflag = False
            
        # implementation
        if gesture == Gest.V_GEST:
            Controller.flag = True
            pyautogui.moveTo(x, y, duration = 0.1)
        elif gesture == Gest.FIST:
            if not Controller.grabflag : 
                Controller.grabflag = True
                pyautogui.mouseDown(button = "left")
            pyautogui.moveTo(x, y, duration = 0.1)
        elif gesture == Gest.MID and Controller.flag:
            pyautogui.click()
            Controller.flag = False
        elif gesture == Gest.INDEX and Controller.flag:
            if time.time() - Controller.last_right_click >1:
                pyautogui.click(button='right')
                Controller.last_right_click = time.time()
            Controller.flag = False
        elif gesture == Gest.TWO_FINGER_CLOSED and Controller.flag:
            pyautogui.doubleClick()
            Controller.flag = False
        elif gesture == Gest.PINCH_MINOR:
            if Controller.pinchminorflag == False:
                Controller.pinch_control_init(hand_result)
                Controller.pinchminorflag = True
            Controller.pinch_control(hand_result,Controller.scrollHorizontal, Controller.scrollVertical)      
        elif gesture == Gest.PINCH_MAJOR:
            if Controller.pinchmajorflag == False:
                Controller.pinch_control_init(hand_result)
                Controller.pinchmajorflag = True
            Controller.pinch_control(hand_result,Controller.changesystembrightness, Controller.changesystemvolume)

In [7]:
# class GestureController:
#     gc_mode = 0
#     cap = None
#     CAM_HEIGHT = None
#     CAM_WIDTH = None
#     hr_major = None # Right Hand by default
#     hr_minor = None # Left hand by default
#     dom_hand = True

#     def __init__(self):
#         GestureController.gc_mode = 1
#         GestureController.cap = cv2.VideoCapture(0)
#         GestureController.CAM_HEIGHT = GestureController.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
#         GestureController.CAM_WIDTH = GestureController.cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    
#     def classify_hands(results):
#         left , right = None,None
#         try:
#             handedness_dict = MessageToDict(results.multi_handedness[0])
#             if handedness_dict['classification'][0]['label'] == 'Right':
#                 right = results.multi_hand_landmarks[0]
#             else :
#                 left = results.multi_hand_landmarks[0]
#         except:
#             pass
#         try:
#             handedness_dict = MessageToDict(results.multi_handedness[1])
#             if handedness_dict['classification'][0]['label'] == 'Right':
#                 right = results.multi_hand_landmarks[1]
#             else :
#                 left = results.multi_hand_landmarks[1]
#         except:
#             pass      
#         if GestureController.dom_hand == True:
#             GestureController.hr_major = right
#             GestureController.hr_minor = left
#         else :
#             GestureController.hr_major = left
#             GestureController.hr_minor = right

#     def start(self):     
#         handmajor = HandRecog(HLabel.MAJOR)
#         handminor = HandRecog(HLabel.MINOR)

#         with mp_hands.Hands(max_num_hands = 2,min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
#             while GestureController.cap.isOpened() and GestureController.gc_mode:
#                 success, image = GestureController.cap.read()

#                 if not success:
#                     print("Ignoring empty camera frame.")
#                     continue              
#                 image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
#                 image.flags.writeable = False
#                 results = hands.process(image)          
#                 image.flags.writeable = True
#                 image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#                 if results.multi_hand_landmarks:                   
#                     GestureController.classify_hands(results)
#                     handmajor.update_hand_result(GestureController.hr_major)
#                     handminor.update_hand_result(GestureController.hr_minor)
#                     handmajor.set_finger_state()
#                     handminor.set_finger_state()
#                     gest_name = handminor.get_gesture()
#                     if gest_name == Gest.PINCH_MINOR:
#                         Controller.handle_controls(gest_name, handminor.hand_result)
#                     else:
#                         gest_name = handmajor.get_gesture()
#                         Controller.handle_controls(gest_name, handmajor.hand_result)
                    
#                     for hand_landmarks in results.multi_hand_landmarks:
#                         mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
#                 else:
#                     Controller.prev_hand = None
#                 cv2.imshow('Virtual Mouse Gesture Controller', image)
#                 if cv2.waitKey(5) & 0xFF == 13:
#                     break
#         GestureController.cap.release()
#         cv2.destroyAllWindows()

In [8]:
class GestureController:
    # Class variables
    gc_mode = 0
    cap = None
    CAM_HEIGHT = None
    CAM_WIDTH = None
    hr_major = None  # Right Hand by default
    hr_minor = None  # Left hand by default 
    dom_hand = True

    def __init__(self):
        """Initialize camera and window settings"""
        GestureController.gc_mode = 1
        GestureController.cap = cv2.VideoCapture(0)
        
        if not GestureController.cap.isOpened():
            print("❌ Webcam could not be opened.")
            return
            
        GestureController.CAM_HEIGHT = GestureController.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        GestureController.CAM_WIDTH = GestureController.cap.get(cv2.CAP_PROP_FRAME_WIDTH)

    @staticmethod
    def classify_hands(results):
        """Classify detected hands as left or right"""
        left, right = None, None
        try:
            # Process first hand
            handedness_dict = MessageToDict(results.multi_handedness[0])
            if handedness_dict['classification'][0]['label'] == 'Right':
                right = results.multi_hand_landmarks[0]
            else:
                left = results.multi_hand_landmarks[0]
        except:
            pass
            
        try:
            # Process second hand if present
            handedness_dict = MessageToDict(results.multi_handedness[1])
            if handedness_dict['classification'][0]['label'] == 'Right':
                right = results.multi_hand_landmarks[1]
            else:
                left = results.multi_hand_landmarks[1]
        except:
            pass

        # Assign hands based on dominance setting
        if GestureController.dom_hand:
            GestureController.hr_major = right
            GestureController.hr_minor = left
        else:
            GestureController.hr_major = left
            GestureController.hr_minor = right

    def start(self):
        """Main loop for hand gesture detection and control"""
        handmajor = HandRecog(HLabel.MAJOR)
        handminor = HandRecog(HLabel.MINOR)

        with mp_hands.Hands(max_num_hands=2, 
                          min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as hands:
            while GestureController.cap.isOpened() and GestureController.gc_mode:
                success, image = GestureController.cap.read()
                if not success:
                    print("Ignoring empty camera frame.")
                    continue
                    
                # Process image
                image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = hands.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # Handle detected hands
                if results.multi_hand_landmarks:
                    GestureController.classify_hands(results)
                    handmajor.update_hand_result(GestureController.hr_major)
                    handminor.update_hand_result(GestureController.hr_minor)
                    
                    # Process gestures
                    handmajor.set_finger_state()
                    handminor.set_finger_state()
                    gest_name = handminor.get_gesture()
                    
                    # Handle controls based on detected gestures
                    if gest_name == Gest.PINCH_MINOR:
                        Controller.handle_controls(gest_name, handminor.hand_result)
                    else:
                        gest_name = handmajor.get_gesture()
                        Controller.handle_controls(gest_name, handmajor.hand_result)

                    # Draw hand landmarks
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(image, hand_landmarks, 
                                               mp_hands.HAND_CONNECTIONS)
                else:
                    Controller.prev_hand = None

                # Display output
                cv2.imshow('Virtual Mouse Gesture Controller', image)
                if cv2.waitKey(5) & 0xFF == 13:  # Exit on Enter key
                    break

        # Cleanup
        GestureController.cap.release()
        cv2.destroyAllWindows()

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk

def runvirtualmouse():
    """Function to start the virtual mouse gesture controller"""
    gc1 = GestureController()
    gc1.start()

# Create main window
root = tk.Tk()
root.geometry("400x300")
root.title("Virtual Mouse Controller")

# Welcome label
label = tk.Label(root, text="Welcome to Virtual Mouse", 
                fg="brown", font='TkDefaultFont 16 bold')
label.grid(row=0, columnspan=5, pady=10, padx=10)

# Image display
try:
    image = ImageTk.PhotoImage(Image.open("tap.png"))
    img_label = tk.Label(root, image=image, width=100, height=100, 
                        borderwidth=3, relief="solid")
    img_label.grid(row=1, columnspan=5, pady=10, padx=10)
except FileNotFoundError:
    # Fallback if image not found
    img_label = tk.Label(root, text="[Image]", width=15, height=6, 
                        borderwidth=3, relief="solid", bg="lightgray")
    img_label.grid(row=1, columnspan=5, pady=10, padx=10)

# Start button
start_button = tk.Button(root, text="Track Mouse", fg="white", bg='green', 
                        font='Helvetica 12 bold italic', command=runvirtualmouse, 
                        height=4, width=16, activebackground='lightblue')
start_button.grid(row=3, column=2, pady=10, padx=20)

# Start the GUI
root.mainloop()